In [ ]:
!pip install transformers dataset accelerate torch
from datasets import load_dataset
!pip install -U transformers

In [ ]:
!git clone https://github.com/palaksriv/PRODIGY-GA-01.git

In [ ]:
!ls

In [ ]:
!ls PRODIGY-GA-01/data

In [ ]:
dataset=load_dataset('text',data_files={ 'train':"PRODIGY-GA-01/data/train.txt"}) #helps in batching
#rather than having it done manually

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
model_name='gpt2'
tokenizer=GPT2Tokenizer.from_pretrained(model_name)
model=GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
dataset

In [ ]:
def tokenize(sample):
  return tokenizer(
      sample['text'],
      truncation=True,
      padding='max_length',
      max_length=128
  )

In [ ]:
tokenized_dataset=dataset.map(tokenize,batched=True,remove_columns=['text'])

In [ ]:
tokenized_dataset

###training

In [ ]:
from transformers import Trainer, TrainingArguments
training_args=TrainingArguments(
    output_dir='/results',
    num_train_epochs=2,
    per_device_train_batch_size=2,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
)


In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    data_collator=data_collator
)
trainer.train()

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id


In [ ]:
prompt = "<|startoftext|>The night was unusually quiet"
input_ids=tokenizer.encode(prompt,return_tensors='pt')
input_ids = input_ids.to(model.device) #move input_ids to the same device as the model
attention_mask = input_ids.ne(tokenizer.pad_token_id).long()
output=model.generate(input_ids=input_ids,attention_mask=attention_mask,max_length=150,temperature=0.8,top_p=0.95,do_sample=True,eos_token_id=tokenizer.eos_token_id,pad_token_id=tokenizer.pad_token_id)
generated_text=tokenizer.decode(output[0],skip_special_tokens=True)
print(generated_text)

In [ ]:
with open('generated_story.txt','w') as f:
  f.write(generated_text)